# further pretraining LUKE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets
!pip install tqdm

In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
import torch
from transformers import AdamW
import datasets


In [ ]:
'''

# Load your custom dataset
dataset_name = "eurlex"
dataset = load_dataset(dataset_name)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Check the type of the dataset
print(type(train_dataset))
print(type(test_dataset))
,,,

In [ ]:
dataset = datasets.load_from_disk("/content/drive/MyDrive/eurlex_dataset/eurlex_train")

In [ ]:
print(dataset)

Dataset({
    features: ['celex_id', 'title', 'text', 'eurovoc_concepts'],
    num_rows: 45000
})


In [ ]:
# List to store dictionaries for each row
dataset_list = []

# Iterate over the dataset
for example in dataset:
    # Create a dictionary for the current row
    row_dict = {
        'celex_id': example['celex_id'],
        'title': example['title'],
        'text': example['text'],
        'eurovoc_concepts': example['eurovoc_concepts']
    }

    # Append the dictionary to the list
    dataset_list.append(row_dict)


In [ ]:
print(len(dataset_list))

45000


In [ ]:
# Calculate the split index
split_index = len(dataset_list) // 2

# Divide the dataset list into two equal parts
dataset_part1 = dataset_list[:split_index]
dataset_part2 = dataset_list[split_index:]

# Print the number of rows in each part
print("Number of rows in part 1:", len(dataset_part1))
print("Number of rows in part 2:", len(dataset_part2))

Number of rows in part 1: 22500
Number of rows in part 2: 22500


In [ ]:
print(split_index)

22500


In [ ]:
from transformers import LukeTokenizer, LukeForMaskedLM, LukeConfig
from torch.utils.data import DataLoader

In [ ]:
# Create a LukeTokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import LukeTokenizer

# Load LukeTokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base")

# Check if the tokenizer has a "mask_token_id"
if hasattr(tokenizer, "mask_token_id"):
    print("The tokenizer has a 'mask_token_id':", tokenizer.mask_token_id)
else:
    print("The tokenizer does not have a 'mask_token_id'")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The tokenizer has a 'mask_token_id': 50264


In [ ]:
# Print the first few samples in the train_dataset
#dataset_part1
#dataset_part2

for i in range(5):
    print(dataset_part2[i])

{'celex_id': '32006R0427', 'title': 'Commission Regulation (EC) No\xa0427/2006 of  15 March 2006  establishing the standard import values for determining the entry price of certain fruit and vegetables\n', 'text': '16.3.2006 EN Official Journal of the European Union L 79/2\nCOMMISSION REGULATION (EC) No 427/2006\nof 15 March 2006\nestablishing the standard import values for determining the entry price of certain fruit and vegetables\nTHE COMMISSION OF THE EUROPEAN COMMUNITIES\n,\nHaving regard to the Treaty establishing the European Community,\nHaving regard to Commission Regulation (EC) No 3223/94 of 21 December 1994 on detailed rules for the application of the import arrangements for fruit and vegetables\xa0(1), and in particular Article 4(1) thereof,\nWhereas:\n(1) Regulation (EC) No 3223/94 lays down, pursuant to the outcome of the Uruguay Round multilateral trade negotiations, the criteria whereby the Commission fixes the standard values for imports from third countries, in respec

In [ ]:
# Access the text data from the dataset
texts_train = [sample["text"] for sample in dataset_part2]


# Tokenize your text data
tokenized_data = tokenizer(texts_train, padding=True, truncation=True, return_tensors="pt")


In [ ]:
# Assuming tokenized_data contains your tokenized data
for i in range(5):  # Print the first 5 tokenized samples
    print(f"Sample {i + 1} - Input IDs: {tokenized_data['input_ids'][i]}")
    print(f"Sample {i + 1} - Attention Mask: {tokenized_data['attention_mask'][i]}")
    print()

Sample 1 - Input IDs: tensor([    0,  1549,     4,   246,     4, 32701, 13245, 13485,  3642,     9,
            5,   796,  1332,   226,  7589,    73,   176, 50118, 10370,   448,
        41336, 26747,  6597,  6034,    36,  3586,    43,   440, 41518,    73,
        32701, 50118,  1116,   379,   494,  3503, 50118, 30248,   154,     5,
         2526,  6595,  3266,    13, 13684,     5,  3555,   425,     9,  1402,
         6231,     8,  8942, 50118, 13354, 31299, 41336,  3243,  1941, 10353,
        23075,  1889, 37275, 26824, 50118,     6, 50118, 15852,  6203,     7,
            5, 20704, 10584,     5,   796,  2573,     6, 50118, 15852,  6203,
            7,  1463, 18912,    36,  3586,    43,   440,   155, 27996,    73,
         6405,     9,   733,   719,  8148,    15,  4271,  1492,    13,     5,
         2502,     9,     5,  6595,  7863,    13,  6231,     8,  8942, 50141,
         1640,   134,   238,     8,    11,  1989,  6776,   204,  1640,   134,
           43, 25991,     6, 50118, 45210,

In [ ]:
# Create PyTorch Dataset using CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
        }


In [ ]:
# Create PyTorch Dataset using CustomDataset class
custom_dataset = CustomDataset(tokenized_data["input_ids"], tokenized_data["attention_mask"])

In [ ]:
# Define your batch size
your_batch_size = 16
learning_rate = 1e-5

In [ ]:
# Create a DataLoader
data_loader = DataLoader(custom_dataset, batch_size=your_batch_size, shuffle=True)

In [ ]:
# Define Luke model configuration and load the pre-trained LukeForMaskedLM model
config = LukeConfig.from_pretrained("studio-ousia/luke-base")
model = LukeForMaskedLM.from_pretrained("studio-ousia/luke-base", config=config)

In [ ]:
# Load the previously saved checkpoint
checkpoint_path = "/content/drive/MyDrive/NLP/luke_checkpoints/luke_model_checkpoint_epoch_11.pth"
checkpoint = torch.load(checkpoint_path)

# Load the model state_dict
model.load_state_dict(checkpoint)

# Set the model to training mode
model.train()

LukeForMaskedLM(
  (luke): LukeModel(
    (embeddings): LukeEmbeddings(
      (word_embeddings): Embedding(50267, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (entity_embeddings): LukeEntityEmbeddings(
      (entity_embeddings): Embedding(500000, 256, padding_idx=0)
      (entity_embedding_dense): Linear(in_features=256, out_features=768, bias=False)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LukeEncoder(
      (layer): ModuleList(
        (0-11): 12 x LukeLayer(
          (attention): LukeAttention(
            (self): LukeSelfAttention(
              (query): Linear(in_features

In [ ]:
# Train the model
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm
import os

# Train the model
num_epochs = 11 # Set the number of epochs 40
mask_probability = 0.15
accumulation_steps = 4  # Define the accumulation steps
checkpoint_dir = "/content/drive/MyDrive/NLP/luke_checkpoints"  # Specify the directory to save the checkpoints

# Create the directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(1, num_epochs + 1):
    for batch_idx, batch in enumerate(tqdm(data_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")):
        inputs = batch["input_ids"]
        attention_mask = batch["attention_mask"]

        mask_indices = torch.rand(inputs.shape) < mask_probability
        labels = inputs.clone()

        inputs[mask_indices] = tokenizer.mask_token_id

        outputs = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss = loss / accumulation_steps
        loss.backward()

        if (batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    # Save model checkpoint after each epoch with epoch number in the file name
    checkpoint_path = os.path.join(checkpoint_dir, f"luke_model_checkpoint_epoch_{epoch + 1}.pth")
    torch.save(model.state_dict(), checkpoint_path)
    # Save the further pretrained model
    model.save_pretrained("/content/drive/MyDrive/NLP/luke_eurlex_model")


In [ ]:
#To later load the pretrained Luke model for downstream tasks

from transformers import LukeForMaskedLM, LukeTokenizer

# Load LukeTokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base")

# Load the saved model
model = LukeForMaskedLM.from_pretrained("/content/drive/MyDrive/NLP/luke_eurlex_model")

# Now you can use the loaded model for downstream tasks
# For example, you can use it for text generation, classification, etc.
